<a href="https://colab.research.google.com/github/mr-osiris/my-new-pern-app/blob/main/Coin_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install scikit-learn
!pip install opencv-python-headless
!pip install joblib
!pip install scikit-image

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import all dependencies
import cv2
import numpy as np
import os
import joblib
from skimage.feature import graycomatrix, graycoprops
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode
import time

print("Setup complete. Libraries installed and Google Drive mounted.")

In [ ]:
def load_dataset_from_drive(drive_path='/content/drive/MyDrive/Indian-Coin-Currency-Dataset'):
    """
    Loads images and labels from a Google Drive folder.
    Handles FileNotFoundError gracefully.
    """
    images = []
    labels = []

    try:
        class_names = sorted([d for d in os.listdir(drive_path) if os.path.isdir(os.path.join(drive_path, d))])

        if not class_names:
            print("Error: No subdirectories found in the dataset path. Make sure your dataset is in folders per class.")
            return images, labels, class_names

        print("Found the following classes:", class_names)

        for class_name in class_names:
            class_path = os.path.join(drive_path, class_name)
            for img_name in os.listdir(class_path):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_path, img_name)
                    img = cv2.imread(img_path)
                    if img is not None:
                        images.append(img)
                        labels.append(class_name)
    except FileNotFoundError:
        print(f"Error: The directory '{drive_path}' was not found. Please check the path and try again.")
        return images, labels, []

    return images, labels, class_names

def preprocess_image(img):
    """
    Using Image Representation & Preprocessing (Unit I)
    Converts image to grayscale, applies a Gaussian filter, and Otsu's thresholding.
    """
    if img is None or img.size == 0:
        return None, None

    # Resize for consistency
    resized_img = cv2.resize(img, (200, 200), interpolation=cv2.INTER_AREA)

    # Grayscale Conversion
    gray = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

    # Gaussian Filter for noise reduction
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Binary Images: Otsu's Thresholding
    _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return resized_img, binary

def extract_features(original_img, binary_img):
    """
    Using Shape and Texture Features (Unit I)
    Extracts shape and texture features from an image.
    """
    features = []

    # Shape features
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the largest contour, assumed to be the coin
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        area = cv2.contourArea(largest_contour)
        perimeter = cv2.arcLength(largest_contour, True)

        circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0

        (x, y), radius = cv2.minEnclosingCircle(largest_contour)

        features.extend([area, perimeter, circularity, radius])
    else:
        features.extend([0, 0, 0, 0])

    # Texture features (GLCM)
    gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)

    # Compute GLCM
    glcm = graycomatrix(gray, distances=[5], angles=[0], levels=256, symmetric=True, normed=True)

    # Extract GLCM properties
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]

    features.extend([contrast, homogeneity, energy])

    return np.array(features)

def train_model(X, y):
    """
    Using PCA and Classifier (Unit II)
    Applies PCA and trains a k-NN classifier.
    Returns the trained model and PCA components.
    """
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Scale the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=0.95)
    X_train_pca = pca.fit_transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    # Train k-NN classifier
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train_pca, y_train)

    # Evaluate and print results
    y_pred = knn.predict(X_test_pca)
    print("Static Test Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

    # Save the model and PCA scaler
    joblib.dump(knn, 'knn_model.joblib')
    joblib.dump(scaler, 'scaler.joblib')
    joblib.dump(pca, 'pca.joblib')

    return knn, scaler, pca

def predict_coin(img, model, scaler, pca):
    """Preprocesses a single image and predicts the coin class."""
    preprocessed_img, binary_img = preprocess_image(img)
    if preprocessed_img is None:
        return "No coin detected", 0.0

    features = extract_features(preprocessed_img, binary_img)

    # Reshape and scale
    features_scaled = scaler.transform(features.reshape(1, -1))

    # Apply PCA
    features_pca = pca.transform(features_scaled)

    # Predict and get probabilities
    prediction = model.predict(features_pca)[0]
    probabilities = model.predict_proba(features_pca)[0]
    confidence = np.max(probabilities)

    return prediction, confidence

In [ ]:
if __name__ == '__main__':
    # Step 1: Dataset Handling
    print("Loading dataset from Google Drive...")
    images, labels, class_names = load_dataset_from_drive()
    print(f"Found {len(images)} images across {len(class_names)} classes.")

    if not images:
        print("No images found. Please check your dataset path and file contents.")
    else:
        # Step 2: Feature Extraction
        print("Extracting features from images...")
        X = []
        y = []
        for img, label in zip(images, labels):
            preprocessed_img, binary_img = preprocess_image(img)
            if preprocessed_img is not None:
                features = extract_features(preprocessed_img, binary_img)
                X.append(features)
                y.append(label)

        X = np.array(X)
        y = np.array(y)

        # Step 3: Model Training
        print("Training model...")
        knn_model, scaler, pca = train_model(X, y)
        print("Model trained and saved to disk.")

In [ ]:
from google.colab import files
import cv2
import numpy as np
import joblib
from IPython.display import Image, display

def predict_uploaded_image():
    """Prompts user to upload an image and predicts the coin class."""

    # Load saved models
    try:
        knn = joblib.load('knn_model.joblib')
        scaler = joblib.load('scaler.joblib')
        pca = joblib.load('pca.joblib')
    except FileNotFoundError:
        print("Models not found. Please run the training script in Cell 3 first.")
        return

    print("Please upload an image file of a coin...")
    uploaded = files.upload()

    if not uploaded:
        print("No file was uploaded.")
        return

    # Process each uploaded file
    for filename in uploaded.keys():
        print(f"Processing '{filename}'...")

        # Read image from uploaded bytes
        img_bytes = uploaded[filename]
        np_arr = np.frombuffer(img_bytes, np.uint8)
        img = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)

        if img is None:
            print(f"Error: Could not decode image from '{filename}'.")
            continue

        # Predict the coin
        prediction, confidence = predict_coin(img, knn, scaler, pca)

        # Overlay the prediction on the image
        label = f"Prediction: {prediction} (Confidence: {confidence:.2f})"
        cv2.putText(img, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Display the result
        print(label)
        _, jpeg = cv2.imencode('.jpg', img)
        display(Image(data=jpeg.tobytes()))

# Run the static image test
predict_uploaded_image()